In [2]:
import numpy as np
import scipy as sp
import igl
import trimesh
import polyscope as ps
import os

root_folder = os.getcwd()

### TODO? Make the demos interactive (e.g. select source vertices for heat geodisics)

In [3]:
[V,F]= igl.read_triangle_mesh(os.path.join(
            root_folder, "../data", "spot.obj"))

In [4]:
#init poly scope and show mesh
ps.init()
ps.set_up_dir('z_up')
ps.set_ground_plane_mode('tile_reflection')

#register mesh
bunny = ps.register_surface_mesh("bunny", V, F, color=[0.6, 0.4, 1.0, 1.0], edge_width=0.9, edge_color=[0.2, 0.1, 0.8, 1.0])
#ps.show()

# Solve Poisson equation (interpolation)

## Mesh Smoothing

In [5]:
# use cotan Laplace matrix for smoothing

L = igl.cotmatrix(V, F)
h = igl.avg_edge_length(V, F) * 0.1

t = 0
n = 15

def smooth():
    global t
    global n
    global V
    global bunny

    if t < n:
        M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
        A = M - h * L

        for i in range(3):
            V[:,i] = sp.sparse.linalg.spsolve(A, M.dot(V[:,i])) # Todo look for a way to use a positive semi-definite solver

        bunny.update_vertex_positions(V)
        t += 1
        print("Smoothing iteration: ", t)

ps.set_user_callback(smooth)

ps.show()

ps.clear_user_callback()



Smoothing iteration:  1
Smoothing iteration:  2
Smoothing iteration:  3
Smoothing iteration:  4
Smoothing iteration:  5
Smoothing iteration:  6
Smoothing iteration:  7
Smoothing iteration:  8
Smoothing iteration:  9
Smoothing iteration:  10
Smoothing iteration:  11
Smoothing iteration:  12
Smoothing iteration:  13
Smoothing iteration:  14
Smoothing iteration:  15


In [27]:
# L = igl.cotmatrix(V, F)
# M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
# h = 0.005
# A = M + h * L

# print(sp.sparse.linalg.norm(M))
# I = sp.sparse.identity(M.shape[0])
# print(I.dot(V[:, 0]) - V[:, 0])
# print(sp.sparse.linalg.norm(sp.sparse.identity(M.shape[0])))


0.08031036303393863
[0. 0. 0. ... 0. 0. 0.]
119.54078801814885
